In [46]:
import os
import pandas as pd
import numpy as np
import glob

In [47]:
movement_path = './pads-project-main/data/movement/'
tabular_path = './pads-project-main/data/ptables/'

X_movement, X_tabular, Y = [], [], []

In [48]:
# Check the dataset pathes
print("📂 Sample movement files:")
print(sorted(os.listdir(movement_path))[:5])

print("\n📂 Sample tabular files:")
print(sorted(os.listdir(tabular_path))[:5])


📂 Sample movement files:
['001_ml.bin', '002_ml.bin', '003_ml.bin', '004_ml.bin', '005_ml.bin']

📂 Sample tabular files:
['001_tbl_ml.bin', '002_tbl_ml.bin', '003_tbl_ml.bin', '004_tbl_ml.bin', '005_tbl_ml.bin']


In [49]:
for bin_file in glob.glob(f"{movement_path}/*_ml.bin"):
    if "_tbl_" in bin_file:
        continue  # skip tabular files
    p_id = os.path.basename(bin_file).split('_')[0]
    tab_file = f"{tabular_path}/{p_id}_tbl_ml.bin"

    if not os.path.exists(tab_file):
        continue

    # Load movement (sensor) and tabular data
    sensor = np.fromfile(bin_file).reshape(-1, 1)  # reshape if needed
    tabular_df = pd.read_pickle(tab_file)

    label = int(tabular_df["con_lbl"].values[0])  # assuming "condition" is encoded as int at position 2
    
    # Drop non-numeric columns before saving tabular features
    non_numeric = ["p_id", "s_id", "con"]  # keep 'con_lbl' for label
    tabular_numeric = tabular_df.drop(columns=non_numeric).astype('float32').to_numpy().squeeze()

    X_movement.append(sensor.astype('float32'))
    X_tabular.append(tabular_numeric)
    Y.append(label)

    # X_movement.append(sensor)
    # X_tabular.append(tabular_df.to_numpy().squeeze())
    # Y.append(label)


In [50]:
print(f"# Movement samples: {len(X_movement)}")
print(f"# Tabular samples: {len(X_tabular)}")
print(f"# Labels: {len(Y)}")


# Movement samples: 469
# Tabular samples: 469
# Labels: 469


In [53]:
print(type(X_tabular[468]))
print(X_tabular[468])

<class 'numpy.ndarray'>
[  1.  54.  44. 165.  67.   1.   0.   0.  -1.  -1.   0.   1.   0.   0.
   1.   0.   0.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [12]:
import torch
import torch.nn as nn

In [24]:
from torch.utils.data import Dataset, DataLoader

class MultimodalDataset(Dataset):
    def __init__(self, X_movement, X_tabular, Y):
        self.X_movement = [torch.tensor(x, dtype=torch.float32) for x in X_movement]
        self.X_tabular = [torch.tensor(x, dtype=torch.float32) for x in X_tabular]
        self.Y = [torch.tensor(y, dtype=torch.long) for y in Y]

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X_movement[idx], self.X_tabular[idx], self.Y[idx]

# Example usage:
dataset = MultimodalDataset(X_movement, X_tabular, Y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [25]:
class AttentionBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=4, batch_first=True)
        self.norm = nn.LayerNorm(dim)
    
    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        return self.norm(x + attn_out)

class MultimodalModel(nn.Module):
    def __init__(self, input_dim_tabular, seq_len, input_dim_movement):
        super().__init__()
        self.movement_encoder = nn.Sequential(
            nn.Conv1d(input_dim_movement, 32, 5, padding=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(32),
        )
        self.tabular_encoder = nn.Sequential(
            nn.Linear(input_dim_tabular, 64),
            nn.ReLU()
        )

        self.attn_movement = AttentionBlock(32)
        self.attn_tabular = AttentionBlock(64)

        self.fusion = nn.Sequential(
            nn.Linear(32 + 64, 64),
            nn.ReLU(),
            nn.Linear(64, 3)  # 3 classes: Healthy, Parkinson's, Other
        )

    def forward(self, movement, tabular):
        # movement: [B, T, 1] → [B, 1, T]
        movement = movement.permute(0, 2, 1)
        move_feat = self.movement_encoder(movement)  # [B, 32, 32]

        # attention expects [B, Seq, F] → transpose from [B, 32, 32] to [B, 32, 32]
        move_feat = self.attn_movement(move_feat.transpose(1, 2)).mean(dim=1)  # [B, 32]

        tab_feat = self.tabular_encoder(tabular).unsqueeze(1)  # [B, 1, 64]
        tab_feat = self.attn_tabular(tab_feat).squeeze(1)      # [B, 64]

        fused = torch.cat([move_feat, tab_feat], dim=1)        # [B, 96]
        return self.fusion(fused)


In [15]:
input_dim_tabular = X_tabular[0].shape[0]  # Or check dynamically via code
model = MultimodalModel(
    input_dim_tabular=input_dim_tabular,  # example
    seq_len=64416,
    input_dim_movement=1   # ✅ Make sure this is 1!
)


In [16]:
print("Input to Conv1d:", movement.permute(0, 2, 1).shape)

NameError: name 'movement' is not defined

In [110]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [111]:
for epoch in range(20):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in dataloader:
        movement, tabular, labels = batch

        # Forward pass
        output = model(movement, tabular)
        loss = criterion(output, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Accumulate loss
        total_loss += loss.item()

        # Compute accuracy
        preds = torch.argmax(output, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total * 100

    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")


Epoch 1 - Loss: 1.0187 | Accuracy: 57.57%
Epoch 2 - Loss: 0.9723 | Accuracy: 58.85%
Epoch 3 - Loss: 0.9553 | Accuracy: 58.85%
Epoch 4 - Loss: 0.9481 | Accuracy: 58.85%
Epoch 5 - Loss: 0.9536 | Accuracy: 58.85%
Epoch 6 - Loss: 0.9455 | Accuracy: 58.85%
Epoch 7 - Loss: 0.9466 | Accuracy: 58.85%
Epoch 8 - Loss: 0.9419 | Accuracy: 58.85%
Epoch 9 - Loss: 0.9383 | Accuracy: 58.85%
Epoch 10 - Loss: 0.9458 | Accuracy: 58.85%
Epoch 11 - Loss: 0.9407 | Accuracy: 58.85%
Epoch 12 - Loss: 0.9339 | Accuracy: 58.85%
Epoch 13 - Loss: 0.9320 | Accuracy: 58.85%
Epoch 14 - Loss: 0.9307 | Accuracy: 58.85%
Epoch 15 - Loss: 0.9319 | Accuracy: 58.85%
Epoch 16 - Loss: 0.9304 | Accuracy: 58.85%
Epoch 17 - Loss: 0.9251 | Accuracy: 58.85%
Epoch 18 - Loss: 0.9222 | Accuracy: 58.85%
Epoch 19 - Loss: 0.9140 | Accuracy: 58.85%
Epoch 20 - Loss: 0.9174 | Accuracy: 58.85%


In [112]:
class TS_BERT(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=4)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.pool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):  # x: (B, T, input_dim)
        x = self.embedding(x)  # (B, T, hidden_dim)
        x = self.encoder(x)    # (B, T, hidden_dim)
        x = x.transpose(1, 2)  # (B, hidden_dim, T)
        x = self.pool(x).squeeze(-1)  # (B, hidden_dim)
        return x

In [113]:
class BiLSTMEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, bidirectional=True)

    def forward(self, x):  # x: (B, T, input_dim)
        out, _ = self.lstm(x)
        return out[:, -1, :]  # or use mean pooling: out.mean(dim=1)

In [115]:
movement_encoder = TS_BERT(input_dim=1, hidden_dim=64, num_layers=2)

/opt/conda/envs/env_dl/lib/python3.9/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [117]:
tabular_encoder = nn.Sequential(
    nn.Linear(input_dim_tabular, 64),
    nn.ReLU()
)

classifier = nn.Sequential(
    nn.Linear(64 + 64, 64),
    nn.ReLU(),
    nn.Linear(64, 3)  # 3 classes
)


In [ ]:
movement_encoder = TS_BERT(input_dim=1, hidden_dim=64, num_layers=2)
tabular_encoder = nn.Sequential(nn.Linear(input_dim_tabular, 64), nn.ReLU())
classifier = nn.Sequential(nn.Linear(64 + 64, 64), nn.ReLU(), nn.Linear(64, 3))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    list(movement_encoder.parameters()) +
    list(tabular_encoder.parameters()) +
    list(classifier.parameters()), lr=1e-3)

for epoch in range(20):
    model_loss = 0
    correct = 0
    total = 0
    for movement, tabular, labels in dataloader:
        # Forward pass
        movement_feat = movement_encoder(movement)  # shape [B, 64]
        tabular_feat = tabular_encoder(tabular)
        fused = torch.cat([movement_feat, tabular_feat], dim=1)
        output = classifier(fused)

        # Loss and optimization
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics
        model_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}, Loss: {model_loss:.4f}, Accuracy: {acc:.2f}%")


Epoch 1, Loss: 14.8345, Accuracy: 56.72%
